In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import math
import re
import os

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
"""""
Original author is Magnus Erik Hvass Pedersen

https://github.com/Hvass-Labs/TensorFlow-Tutorials

https://www.youtube.com/watch?v=vI2Y3I-JI2Q&t=1824s&ab_channel=HvassLaboratories

License(MIT)
These tutorials and source-code are published under the MIT License which allows very broad use for both academic and commercial purposes.

A few of the images used for demonstration purposes may be under copyright. These images are included under the "fair usage" laws.

You are very welcome to modify these tutorials and use them in your own projects. Please keep a link to the original repository.
"""

In [3]:
tf.__version__

'2.6.0'

In [4]:
tf.keras.__version__

'2.6.0'

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 254509895096962513
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8809781079984874963
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [6]:
df  = pd.read_csv('en-ig.csv',sep=';')

In [8]:
# type(df["english"].tolist())

In [9]:
language_code='da'

In [10]:
# df = df.to_nump()

In [11]:
mark_start = 'ssss '
mark_end = ' eeee'

In [17]:
data_src = df["yoruba"].tolist()

data_dest = df["english"].tolist()


for i in range(len(data_src)):
    #re.sub('[^a-zA-Z0-9 ]+', '', str(data_src[i]))
    data_src[i]=str(data_src[i])


for i in range(len(data_dest)):
    #data_dest[i]=mark_start+str(data_dest[i])+mark_end
    data_dest[i]=mark_start+str(data_dest[i])+mark_end
    
print(str(data_dest[6]))

ssss Tottenham emerila n'asọmpị iko FA ugboro asatọ eeee


In [19]:
idx = 2

In [20]:
# data_src = data_src[:10000]
# print(len(data_src))

In [21]:
# data_dest = data_dest[:10000]

In [22]:
# idx = 8002

In [23]:
#data_src[idx]

In [24]:
# data_dest[idx]

In [25]:
num_words = 10000

In [26]:
class TokenizerWrap(Tokenizer):
    """Wrap the Tokenizer-class from Keras with more functionality."""
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
        """
        :param texts: List of strings. This is the data-set.
        :param padding: Either 'post' or 'pre' padding.
        :param reverse: Boolean whether to reverse token-lists.
        :param num_words: Max number of words to use.
        """

        Tokenizer.__init__(self, num_words=num_words)

        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)

        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Reverse the token-sequences.
            self.tokens = [list(reversed(x)) for x in self.tokens]
        
            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        # The number of integer-tokens in each sequence.
        self.num_tokens = [len(x) for x in self.tokens]

        # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        # This is a compromise so we save a lot of memory and
        # only have to truncate maybe 5% of all the sequences.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        # Pad / truncate all token-sequences to the given length.
        # This creates a 2-dim numpy matrix that is easier to use.
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        """Lookup a single word from an integer-token."""

        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        """Convert a list of integer-tokens to a string."""

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        # Concatenate the words to a single string
        # with space between all the words.
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text, reverse=False, padding=False):
        """
        Convert a single text-string to tokens with optional
        reversal and padding.
        """

        # Convert to tokens. Note that we assume there is only
        # a single text-string so we wrap it in a list.
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            # Reverse the tokens.
            tokens = np.flip(tokens, axis=1)

            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        if padding:
            # Pad and truncate sequences to the given length.
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

In [27]:
%%time
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=True,
                              num_words=num_words)

Wall time: 293 ms


In [28]:
%%time
tokenizer_dest = TokenizerWrap(texts=data_dest,
                               padding='post',
                               reverse=False,
                               num_words=num_words)

Wall time: 505 ms


In [29]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(10000, 39)
(10000, 45)


In [30]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

2

In [31]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

3

In [32]:
idx = 2

In [33]:
tokens_src[idx]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0, 4002,    5,  232,
        230,  432,   17,   14,   34,  904])

In [35]:
data_src[idx]

'Dapchi: Government has not defeated Boko Haram and Massob'

In [36]:
tokens_dest[idx]

array([   2,  881,  152, 7433,  298,  302, 3211,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [39]:
encoder_input_data = tokens_src

In [40]:
decoder_input_data = tokens_dest[:, :-1]
decoder_input_data.shape

(10000, 44)

In [41]:
decoder_output_data = tokens_dest[:, 1:]
decoder_output_data.shape

(10000, 44)

In [42]:
idx = 2

In [43]:
decoder_input_data[idx]

array([   2,  881,  152, 7433,  298,  302, 3211,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [44]:
decoder_output_data[idx]

array([ 881,  152, 7433,  298,  302, 3211,    3,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [45]:
tokenizer_dest.tokens_to_string(decoder_input_data[idx])

'ssss dapchi gọọmenti emeribeghị boko haram massob eeee'

In [46]:
tokenizer_dest.tokens_to_string(decoder_output_data[idx])

'dapchi gọọmenti emeribeghị boko haram massob eeee'

In [47]:
encoder_input = Input(shape=(None, ), name='encoder_input')

In [48]:
embedding_size = 128

In [49]:
encoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='encoder_embedding')

This is the size of the internal states of the Gated Recurrent Units (GRU). The same size is used in both the encoder and decoder.

In [50]:
state_size = 512

This creates the 3 GRU layers that will map from a sequence of embedding-vectors to a single "thought vector" which summarizes the contents of the input-text. Note that the last GRU-layer does not return a sequence.

In [51]:
encoder_gru1 = GRU(state_size, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2',
                   return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3',
                   return_sequences=False)

This helper-function connects all the layers of the encoder.

In [52]:
def connect_encoder():
    # Start the neural network with its input-layer.
    net = encoder_input
    
    # Connect the embedding-layer.
    net = encoder_embedding(net)

    # Connect all the GRU-layers.
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)

    # This is the output of the encoder.
    encoder_output = net
    
    return encoder_output

In [53]:
encoder_output = connect_encoder()

In [54]:
decoder_initial_state = Input(shape=(state_size,),
                              name='decoder_initial_state')

In [55]:
decoder_input = Input(shape=(None, ), name='decoder_input')

In [56]:
decoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [57]:
decoder_gru1 = GRU(state_size, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3',
                   return_sequences=True)

In [58]:
decoder_dense = Dense(num_words,
                      activation='softmax',
                      name='decoder_output')

In [59]:
def connect_decoder(initial_state):
    # Start the decoder-network with its input-layer.
    net = decoder_input

    # Connect the embedding-layer.
    net = decoder_embedding(net)
    
    # Connect all the GRU-layers.
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    # Connect the final dense layer that converts to
    # one-hot encoded arrays.
    decoder_output = decoder_dense(net)
    
    return decoder_output

In [60]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input],
                    outputs=[decoder_output])

In [61]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])

In [62]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state],
                      outputs=[decoder_output])

In [63]:
model_train.compile(optimizer=RMSprop(learning_rate=1e-3),
                    loss='sparse_categorical_crossentropy')

In [64]:
path_checkpoint = '21_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [65]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=3, verbose=1)

In [66]:
callback_tensorboard = TensorBoard(log_dir='./21_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [67]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard]

In [83]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [69]:
x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

print(len(encoder_input_data))

10000


In [70]:
y_data = \
{
    'decoder_output': decoder_output_data
}

In [71]:
validation_split = 500 / len(encoder_input_data)
validation_split

0.05

In [72]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=90,
                epochs=10,
                validation_split=0.12,
                callbacks=callbacks)

Epoch 1/10
98/98 [==============================] - 29s 226ms/step - loss: 1.6109 - val_loss: 3.1855

Epoch 00001: val_loss improved from inf to 3.18554, saving model to 21_checkpoint.keras
Epoch 2/10
98/98 [==============================] - 21s 215ms/step - loss: 1.5011 - val_loss: 3.2161

Epoch 00002: val_loss did not improve from 3.18554
Epoch 3/10
98/98 [==============================] - 21s 216ms/step - loss: 1.4281 - val_loss: 3.2389

Epoch 00003: val_loss did not improve from 3.18554
Epoch 4/10
98/98 [==============================] - 21s 216ms/step - loss: 1.3638 - val_loss: 3.2509

Epoch 00004: val_loss did not improve from 3.18554
Epoch 00004: early stopping


In [73]:
with open('token_src.pickle','wb') as f:
    pickle.dump(tokenizer_src,f)
    
with open('token_dest.pickle','wb') as f:
    pickle.dump(tokenizer_dest,f)
    
# model_encoder.save("model_e")
# model_decoder.save('model_d')
tf.keras.models.save_model(model_encoder, "model_e2")
tf.keras.models.save_model(model_decoder, "model_d2")
     #tf.keras.models.save_model()

INFO:tensorflow:Assets written to: model_e2\assets


INFO:tensorflow:Assets written to: model_e2\assets


INFO:tensorflow:Assets written to: model_d2\assets


INFO:tensorflow:Assets written to: model_d2\assets


In [74]:
def translate(input_text, true_output_text=None):
    """Translate a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    #     tkn = TokenizerWrap(texts=input_text,
    #                               padding='pre',
    #                               reverse=True)
    
    
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding=True)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    print('THE MODEL IS',model_encoder)
    
    
    initial_state = model_encoder.predict(input_tokens)
    #print('THIS IS THE STATE',initial_state)

    # Max number of tokens / words in the output sequence.
    max_tokens = tokenizer_dest.max_tokens

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens,
    # but the decoder-model expects a batch of sequences.
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0, count_tokens] = token_int

        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
            'decoder_initial_state': initial_state,
            'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict(), but it would make the code
        # much more complicated.

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)

        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0, count_tokens, :]
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens += 1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    
    # Print the input-text.
    print("Input text:")
    print(input_text)
    print()

    # Print the translated output-text.
    print("Translated text:")
    print(output_text)
    print()

    # Optionally print the true translated text.
    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()

### Examples

Translate a text from the training-data. This translation is quite good. Note how it is not identical to the translation from the training-data, but the actual meaning is similar.

In [92]:
idx = 3
translate(input_text=data_src[idx],
          true_output_text=data_dest[idx])

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
Tottenham look forward to lifting the FA cup in defeating Rochadale.

Translated text:
 cheta na mba ofesi na mba ọzọ nwere ike iri iri eeee

True output text:
ssss Tottenham na-ele anya iburu iko FA na mmeri Rochadale eeee



Here is another example which is also a reasonable translation, although it has incorrectly translated the natural disasters. Note "countries of the European Union" has instead been translated as "member states" which are synonyms in this context.

In [94]:
idx = 4
translate(input_text=data_src[idx],
          true_output_text=data_dest[idx])

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
Son Heung-min, Fernando Llorente and Kyle Walter-Peters of Tottenham beat Rochadale mercilessly in their yesterday's FA cup competition. 

Translated text:
 onye ntaakụkọ na ndị mba amerika bụ ahụike n'aha ịchafụ kwuru na ha ga enye ndị mmadụ naịjirịa na enweta ndị naịjirịa na ndị super eagles eeee

True output text:
ssss Son Heung-min, Fernando Llorente na Kyle Walter-Peters nke Tottenham bụ ndị mmechiri ọnụ Rochadale n'asọmpị ụnyaahụ. eeee



In this example we join two texts from the training-set. The model first sends this combined text through the encoder, which produces a "thought-vector" that seems to summarize both texts reasonably well so the decoder can produce a reasonable translation.

In [95]:
idx = 3
translate(input_text=data_src[idx] + data_src[idx+1],
          true_output_text=data_dest[idx] + data_dest[idx+1])

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
Tottenham look forward to lifting the FA cup in defeating Rochadale.Son Heung-min, Fernando Llorente and Kyle Walter-Peters of Tottenham beat Rochadale mercilessly in their yesterday's FA cup competition. 

Translated text:
 n'ime nke a bụ nke nwaanyị na akpọ nke a bụ nke nwaanyị a mụrụ kwuo na anọ a na agba anya na oke a na agba nke afọ a na agba nke afọ a na agba nke ọnwa eeee

True output text:
ssss Tottenham na-ele anya iburu iko FA na mmeri Rochadale eeeessss Son Heung-min, Fernando Llorente na Kyle Walter-Peters nke Tottenham bụ ndị mmechiri ọnụ Rochadale n'asọmpị ụnyaahụ. eeee



If we reverse the order of these two texts then the meaning is not quite so clear for the latter text.

In [87]:
idx = 3
translate(input_text=data_src[idx+1] + data_src[idx],
          true_output_text=data_dest[idx+1] + data_dest[idx])

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
Son Heung-min, Fernando Llorente and Kyle Walter-Peters of Tottenham beat Rochadale mercilessly in their yesterday's FA cup competition. Tottenham look forward to lifting the FA cup in defeating Rochadale.

Translated text:
 onye na emepụta ọnụ na mba amerika bụ ike na ndị na emepụta ọrịa na mba ọrụ ahụ na enye ụmụ egwuregwu bọọlụ iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri iri

True output text:
ssss Son Heung-min, Fernando Llorente na Kyle Walter-Peters nke Tottenham bụ ndị mmechiri ọnụ Rochadale n'asọmpị ụnyaahụ. eeeessss Tottenham na-ele anya iburu iko FA na mmeri Rochadale eeee



This is an example I made up. It is a quite broken translation.

In [88]:
translate(input_text="i like you",
          true_output_text='how are you')

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
i like you

Translated text:
 ihe nkiri eeee

True output text:
how are you



In [89]:
translate(input_text="The news that will interest you",
          true_output_text='how are you')

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
The news that will interest you

Translated text:
 akụkọ ga amasị gị eeee

True output text:
how are you



This is another example I made up. This is a better translation even though it is perhaps a more complicated text.

In [90]:
translate(input_text="They came and arrested some of them.",
          true_output_text="Ha bịara nwụchie ụfọdụ n'ime ha.")

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
They came and arrested some of them.

Translated text:
 ha na agba bọọlụ na if dị na mba swidin eeee

True output text:
Ha bịara nwụchie ụfọdụ n'ime ha.



This is a text from a Danish song. It doesn't even make much sense in Danish. However the translation is probably so broken because several of the words are not in the vocabulary.

In [82]:
translate(input_text="Hvem spæner ud af en butik og tygger de stærkeste bolcher?",
          true_output_text="Who runs out of a shop and chews the strongest bon-bons?")

THE MODEL IS <keras.engine.functional.Functional object at 0x0000019B685A95B0>
Input text:
Hvem spæner ud af en butik og tygger de stærkeste bolcher?

Translated text:
 n'ofesi eeee

True output text:
Who runs out of a shop and chews the strongest bon-bons?

